In [1]:
import json
import pandas as pd
import glob
import xmltodict, json
from pandasql import sqldf

In [2]:
pysqldf = lambda q: sqldf(q, globals())

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [4]:
files = glob.glob('IADownloads/*')

rows = []

for file in files:
    print(file)
    row = []
    
    fname = file.split('/')[1]
    
    if fname != 'Search_IA.ipynb':
    
        with open(file + '/' + file.split('/')[1] + '_meta.xml') as f:

            try:
                jsn = xmltodict.parse(f.read())

                row.append(jsn['metadata']['identifier'])
                row.append(','.join(jsn['metadata']['collection']))
                row.append(jsn['metadata']['mediatype'])
                row.append(jsn['metadata']['title'])
                if 'date' in jsn['metadata']:
                    row.append(jsn['metadata']['date'])
                elif 'year' in jsn['metadata']:
                    row.append(jsn['metadata']['year'])
                else:
                    row.append('')
                if 'description' in jsn['metadata']:
                    row.append(jsn['metadata']['description'])
                else:
                    row.append('')
                row.append(','.join(jsn['metadata']['subject'].split(';')[:-1]))
            except:
                print('error in', file)
        rows.append(row)

IADownloads/tobacco_qno71d00
IADownloads/tobacco_kon79e00
IADownloads/opioids_yfxh0257
IADownloads/opioids_ggxh0257
IADownloads/opioids_kgxh0257
IADownloads/tobacco_coo71d00
IADownloads/tobacco_lxkv0152
IADownloads/opioids_mfxh0257
IADownloads/tobacco_gfk52b00
IADownloads/tobacco_klmf0164
IADownloads/tobacco_wrm44a00
IADownloads/opioids_fzhh0257
IADownloads/tobacco_ptfb0229
IADownloads/tobacco_uek52b00
IADownloads/tobacco_yek52b00
IADownloads/opioids_jzhh0257
IADownloads/tobacco_zeu62i00
IADownloads/opioids_rzhh0257
IADownloads/tobacco_jbj54a00
IADownloads/tobacco_vru47h00
IADownloads/tobacco_kphh0225
IADownloads/opioids_hzhh0257
IADownloads/Tobacco_qyq95i00
IADownloads/tobacco_rtfb0229
IADownloads/tobacco_yxxh0225
IADownloads/tobacco_wek52b00
IADownloads/tobacco_oek52b00
IADownloads/tobacco_xtpp0225
IADownloads/tobacco_myhh0257
IADownloads/opioids_pzhh0257
IADownloads/tobacco_mxxh0225
IADownloads/tobacco_aoo71d00
IADownloads/tobacco_elm09c00
IADownloads/tobacco_sno71d00
IADownloads/to

In [5]:
len(rows[0])

7

In [6]:
df_metadata = pd.DataFrame(rows, columns=['identifier', 'collection', 'title', 'mediatype', 'year', 'description','subject'])

In [7]:
df_metadata

,identifier,collection,title,mediatype,year,description,subject
0,tobacco_qno71d00,"industry-archives,newsandpublicaffairs,medicalheritagelibrary",movies,"People of California v. Philip Morris Incorporated Et Al. Edward Blackmer Deposition Winston-Salem, NC; Tape 2 of 6 [Parts 1-5]",,"Deposition of Edward Blackmer, head of the Marketing Operations Department at R.J. Reynolds, testifying in the People of California, and the Commonwealth of Massachusetts, v. Philip Morris Incorporated Et Al.\n\n<a href=""http://www.archive.org/details/tobacco_qno71d00a"" rel=""nofollow"">Click Here to View Part 1</a>\n\n<a href=""http://www.archive.org/details/tobacco_qno71d00b"" rel=""nofollow"">Click Here to View Part 2</a>\n\n<a href=""http://www.archive.org/details/tobacco_qno71d00c"" rel=""nofollow"">Click Here to View Part 3</a>\n\n<a href=""http://www.archive.org/details/tobacco_qno71d00d"" rel=""nofollow"">Click Here to View Part 4</a>\n\n<a href=""http://www.archive.org/details/tobacco_qno71d00e"" rel=""nofollow"">Click Here to View Part 5</a>","tobacco , cigarette , marketing , advertising , promotions , promotional merchandise , sales , event sponsorship , sports sponsorship , youth , smoking prevention , marketing research , tobacco control , lobbying , cigarette brands , filter , demographics , tar , nicotine , working class , international trade , marketing strategy , cigarette promotion code , low yield cigarette , age group , smoker behavior"
1,tobacco_kon79e00,"industry-archives,newsandpublicaffairs,medicalheritagelibrary",movies,Thomas Osdene Deposition Preparation,1996,"Thomas Osdene, Director of Science and Technology at Philip Morris, practices speaking in front of a group in preparation for his deposition.","tobacco , cigarette , legal activity , lawsuit"
2,opioids_yfxh0257,"industry-archives,newsandpublicaffairs",movies,"Deposition of James Jim Rausch, Customer Service Manager for Finished Goods",2018,Deposition of James Rauch in RE: National Prescription Opiate Litigation; in 8 parts,"lawsuit, legal activity, litigation, opioids, sales, marketing"
3,opioids_ggxh0257,"industry-archives,newsandpublicaffairs",movies,"Deposition of Kevin Vorderstrasse, Senior Director of Product Management and Analytics for Specialty Generics",2018,Deposition of Kevin Vorderstrasse in RE: National Prescription Opiate Litigation; in 12 parts,"lawsuit, legal activity, litigation, opioids, sales, marketing"
4,opioids_kgxh0257,"industry-archives,newsandpublicaffairs",movies,"Deposition of Michael Wessler, Product Director for Marketing",2019,Deposition of Michael Wessler in RE: National Prescription Opiate Litigation; in 3 parts,"lawsuit, legal activity, litigation, opioids, sales, marketing"
...,...,...,...,...,...,...,...
139,tobacco_pvw10j00,"industry-archives,newsandpublicaffairs",movies,"Deposition of ANDREW J. SCHINDLER, April 14, 1997, ENGLE v. R.J. REYNOLDS TOBACCO CO.",1997-04-14,"The witness, CEO and President of R.J. Reynolds Tobacco, was deposed by \nthe plaintiffs. He began by comparing the marketing position of RJR \nbrands to Marlboro, which has been the top selling cigarette for the \npast eighteen years. He described the inadequacies of current scientific\n knowledge, claiming for example that mouse studies have failed to show \ncausality between tobacco and cancer. He also claimed that \nepidemiological research has failed to show environmental tobacco smoke \nto be as a risk factor for cancer. He discussed environmental tobacco \nsmoke within the context of an airline attendant lawsuit and commented \non the Environmental Protection Agency's decision to classify it as a \ncarcinogen. The witness addressed the role of the government in \nregulating environmental tobacco smoke. He also discussed youth \nmarketing, the role of nicotine in attracting new smokers, and the role \nof the Tobacco Institute.","tobacco, cigarette, deposition, litigation, advertising, industry response, youth"
140,tobacco_juz96g00,"industry-archives,newsa

In [8]:
#df_metadata.to_csv('metadata.csv', index=False)

In [9]:
files = glob.glob('IADownloads/*')

rows = []
for file in files:
    
    fname = file.split('/')[1]
    print(file, fname)
    
    if fname != 'Search_IA.ipynb':
    
        with open(file + '/' + fname + '_files.xml') as f:

            jsn = xmltodict.parse(f.read())
            
            #print(jsn.keys())
            
            for f in jsn['files']['file']:  
                #print(f)
                #print('***', f['@name'].split('.')[-1])
                row = []
                if f['@name'].split('.')[-1] == 'mp4':  
                    dl = f['@name']
                    s = int(f['size'])
                    l = float(f['length'])
                    url = 'https://archive.org/download/' + file.split('/')[1] + '/' + dl
                    row.append(file.split('/')[1])
                    row.append(dl)
                    row.append(s)
                    row.append(l)
                    row.append(url)
                    #print(row)
                    rows.append(row)


IADownloads/tobacco_qno71d00 tobacco_qno71d00
IADownloads/tobacco_kon79e00 tobacco_kon79e00
IADownloads/opioids_yfxh0257 opioids_yfxh0257
IADownloads/opioids_ggxh0257 opioids_ggxh0257
IADownloads/opioids_kgxh0257 opioids_kgxh0257
IADownloads/tobacco_coo71d00 tobacco_coo71d00
IADownloads/tobacco_lxkv0152 tobacco_lxkv0152
IADownloads/opioids_mfxh0257 opioids_mfxh0257
IADownloads/tobacco_gfk52b00 tobacco_gfk52b00
IADownloads/tobacco_klmf0164 tobacco_klmf0164
IADownloads/tobacco_wrm44a00 tobacco_wrm44a00
IADownloads/opioids_fzhh0257 opioids_fzhh0257
IADownloads/tobacco_ptfb0229 tobacco_ptfb0229
IADownloads/tobacco_uek52b00 tobacco_uek52b00
IADownloads/tobacco_yek52b00 tobacco_yek52b00
IADownloads/opioids_jzhh0257 opioids_jzhh0257
IADownloads/tobacco_zeu62i00 tobacco_zeu62i00
IADownloads/opioids_rzhh0257 opioids_rzhh0257
IADownloads/tobacco_jbj54a00 tobacco_jbj54a00
IADownloads/tobacco_vru47h00 tobacco_vru47h00
IADownloads/tobacco_kphh0225 tobacco_kphh0225
IADownloads/opioids_hzhh0257 opioi

In [10]:
df_files = pd.DataFrame(rows, columns=['identifier', 'name', 'size', 'length', 'url'])

In [11]:
df = pd.merge(df_files, df_metadata,on='identifier')

In [12]:
df.to_csv('csv_files/metadata_links.csv', index=False)

In [13]:
df.head()

,identifier,name,size,length,url,collection,title,mediatype,year,description,subject
0,tobacco_qno71d00,VTS_01_1_512kb.mp4,125757784,1735.68,https://archive.org/download/tobacco_qno71d00/VTS_01_1_512kb.mp4,"industry-archives,newsandpublicaffairs,medicalheritagelibrary",movies,"People of California v. Philip Morris Incorporated Et Al. Edward Blackmer Deposition Winston-Salem, NC; Tape 2 of 6 [Parts 1-5]",,"Deposition of Edward Blackmer, head of the Marketing Operations Department at R.J. Reynolds, testifying in the People of California, and the Commonwealth of Massachusetts, v. Philip Morris Incorporated Et Al.\n\n<a href=""http://www.archive.org/details/tobacco_qno71d00a"" rel=""nofollow"">Click Here to View Part 1</a>\n\n<a href=""http://www.archive.org/details/tobacco_qno71d00b"" rel=""nofollow"">Click Here to View Part 2</a>\n\n<a href=""http://www.archive.org/details/tobacco_qno71d00c"" rel=""nofollow"">Click Here to View Part 3</a>\n\n<a href=""http://www.archive.org/details/tobacco_qno71d00d"" rel=""nofollow"">Click Here to View Part 4</a>\n\n<a href=""http://www.archive.org/details/tobacco_qno71d00e"" rel=""nofollow"">Click Here to View Part 5</a>","tobacco , cigarette , marketing , advertising , promotions , promotional merchandise , sales , event sponsorship , sports sponsorship , youth , smoking prevention , marketing research , tobacco control , lobbying , cigarette brands , filter , demographics , tar , nicotine , working class , international trade , marketing strategy , cigarette promotion code , low yield cigarette , age group , smoker behavior"
1,tobacco_kon79e00,VTS_01_1_512kb.mp4,50413643,692.66,https://archive.org/download/tobacco_kon79e00/VTS_01_1_512kb.mp4,"industry-archives,newsandpublicaffairs,medicalheritagelibrary",movies,Thomas Osdene Deposition Preparation,1996,"Thomas Osdene, Director of Science and Technology at Philip Morris, practices speaking in front of a group in preparation for his deposition.","tobacco , cigarette , legal activity , lawsuit"
2,opioids_yfxh0257,yfxh0257-0001.ia.mp4,709806303,3563.00,https://archive.org/download/opioids_yfxh0257/yfxh0257-0001.ia.mp4,"industry-archives,newsandpublicaffairs",movies,"Deposition of James Jim Rausch, Customer Service Manager for Finished Goods",2018,Deposition of James Rauch in RE: National Prescription Opiate Litigation; in 8 parts,"lawsuit, legal activity, litigation, opioids, sales, marketing"
3,opioids_yfxh0257,yfxh0257-0001.mp4,709806303,3563.00,https://archive.org/download/opioids_yfxh0257/yfxh0257-0001.mp4,"industry-archives,newsandpublicaffairs",movies,"Deposition of James Jim Rausch, Customer Service Manager for Finished Goods",2018,Deposition of James Rauch in RE: National Prescription Opiate Litigation; in 8 parts,"lawsuit, legal activity, litigation, opioids, sales, marketing"
4,opioids_yfxh0257,yfxh0257-0002.ia.mp4,876775594,4404.11,https://archive.org/download/opioids_yfxh0257/yfxh0257-0002.ia.mp4,"industry-archives,newsandpublicaffairs",movies,"Deposition of James Jim Rausch, Customer Service Manager for Finished Goods",2018,Deposition of James Rauch in RE: National Prescription Opiate Litigation; in 8 parts,"lawsuit, legal activity, litigation, opioids, sales, marketing"


In [14]:
cloud_data_transfer = pysqldf("""
SELECT DISTINCT url FROM df
WHERE url NOT LIKE '%.ia.%'
""")

In [15]:
cloud_data_transfer.columns = ['TsvHttpData-1.0']

In [16]:
cloud_data_transfer

,TsvHttpData-1.0
0,https://archive.org/download/tobacco_qno71d00/VTS_01_1_512kb.mp4
1,https://archive.org/download/tobacco_kon79e00/VTS_01_1_512kb.mp4
2,https://archive.org/download/opioids_yfxh0257/yfxh0257-0001.mp4
3,https://archive.org/download/opioids_yfxh0257/yfxh0257-0002.mp4
4,https://archive.org/download/opioids_yfxh0257/yfxh0257-0003.mp4
...,...
554,https://archive.org/download/opioids_hfxh0257/hfxh0257-0001.mp4
555,https://archive.org/download/opioids_hfxh0257/hfxh0257-0002.mp4
556,https://archive.org/download/opioids_hfxh0257/hfxh0257-0003.mp4
557,https://archive.org/download/opioids_hfxh0257/hfxh0257-0004.mp4


In [17]:
cloud_data_transfer.to_csv('csv_files/cloud_data_transfer.tsv', sep='\t',index=False)